In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.net = nn.Sequential(
            #out = ((Input width - filter size + 2* padding) / Stride) + 1
            nn.Conv2d(1, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(5, 5),
            nn.Conv2d(16, 16, 3),
            nn.Flatten(),
            nn.Linear(144, 120),
            nn.ReLU(),
            nn.Linear(120, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Softmax()
        )
        
    def forward(self, x):
        #x = x.permute(0, 3, 1, 2)
        x = self.net(x)
        return x
    
    def accuracy(self, out, yb):
        preds = torch.argmax(out, dim=1)
        return (preds == yb).float().mean()

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        pass
        #torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

params = {'batch_size': 64,
          'shuffle': True}
max_epochs = 200

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
        
model = CNN()
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)


model.cuda()
model.apply(weights_init)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.0)


train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=64, shuffle=True)

start.record()
for epochs in range(max_epochs):
    # Training
    correct = 0
    total = 0
    print("start epoch. Correct:", correct, "total: ", total)
    for batch_idx, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        target = target.to(device)

        data = np.repeat(data[..., np.newaxis], 3, -1)
        
        prediction = model.forward(data[:, 0, :, : ,:].permute(0, 3, 1, 2).to(device))
        argmax_predicton = torch.argmax(prediction, dim = 1)
    
        loss = nn.functional.cross_entropy(prediction, target)
        
        total += target.size(0)
        
        correct += (argmax_predicton == target).sum().item()
        
        loss.backward()
        optimizer.step()
            
    print(f'Epoch: [{epochs+1}/{max_epochs}], Loss: {loss.item():.4f}, Acc: {correct} / {total}')

writer.close()
end.record()
# Waits for everything to finish running
torch.cuda.synchronize()

print(start.elapsed_time(end)/1000/60, "min")

Using cuda device
start epoch. Correct: 0 total:  0


Using cache found in C:\Users\bruno/.cache\torch\hub\pytorch_vision_v0.10.0


Epoch: [1/200], Loss: 0.5923, Acc: 57763 / 60000
start epoch. Correct: 0 total:  0
Epoch: [2/200], Loss: 0.0086, Acc: 59318 / 60000
start epoch. Correct: 0 total:  0
Epoch: [3/200], Loss: 0.0021, Acc: 59559 / 60000
start epoch. Correct: 0 total:  0
Epoch: [4/200], Loss: 0.0015, Acc: 59684 / 60000
start epoch. Correct: 0 total:  0
Epoch: [5/200], Loss: 0.1429, Acc: 59795 / 60000
start epoch. Correct: 0 total:  0
Epoch: [6/200], Loss: 0.0052, Acc: 59829 / 60000
start epoch. Correct: 0 total:  0
Epoch: [7/200], Loss: 0.0003, Acc: 59860 / 60000
start epoch. Correct: 0 total:  0
Epoch: [8/200], Loss: 0.0004, Acc: 59903 / 60000
start epoch. Correct: 0 total:  0
Epoch: [9/200], Loss: 0.0010, Acc: 59908 / 60000
start epoch. Correct: 0 total:  0
Epoch: [10/200], Loss: 0.0003, Acc: 59933 / 60000
start epoch. Correct: 0 total:  0
Epoch: [11/200], Loss: 0.0014, Acc: 59954 / 60000
start epoch. Correct: 0 total:  0
Epoch: [12/200], Loss: 0.0040, Acc: 59952 / 60000
start epoch. Correct: 0 total:  0
E

KeyboardInterrupt: 

In [2]:
%%time

from os import path

import numpy as np
import pandas as pd
import time

import torch
from torch.utils.data import Dataset, DataLoader, BatchSampler, Sampler
from torch import nn
import torchvision
from torchvision import datasets, transforms, models

device = torch.device("cuda")
print('Using {} device'.format(device))

Using cuda device
Wall time: 1.97 s


In [3]:
#out = ((Input width - filter size + 2* padding) / Stride) + 1
#dropout = 0 is shortcutted
class CNN(nn.Module):
    def __init__(self, img_shape=128, label_shape=8, channel_numbers=[3,16], number_of_c_layers = 2, number_of_filters=[16,16], 
                 kernel_sizes=[3,3], strides=[1,1], paddings=[0,0], fclayout = [120, 32], maxpool = [4, 4], device = "cuda", BN=False, dropout_rate = 0):
        super(CNN, self).__init__()
        
        self.device = device
        self.channel_number = channel_numbers
        self.number_of_filters = number_of_filters
        self.kernel_sizes = kernel_sizes
        self.strides = strides
        self.paddings = paddings
        self.maxpool = maxpool
        self.fclayout = fclayout
        self.fcs = []
        self.label_shape = label_shape
        self.img_shape = img_shape
        self.dropout_rate = dropout_rate
            
        self.net = nn.Sequential()
        
        for i in range(number_of_c_layers): 
            if i == number_of_c_layers-1:
                self.net.add_module("conv2-" + str(i), nn.Conv2d(channel_numbers[i], number_of_filters[i], kernel_sizes[i], stride=strides[i], padding=paddings[i]))
                if BN:
                    self.net.add_module("BN-2d-" + str(i), nn.BatchNorm2d(number_of_filters[i]))
                self.net.add_module("relu-conv-" + str(i), nn.ReLU())
            else:
                self.net.add_module("conv2-" + str(i), nn.Conv2d(channel_numbers[i], number_of_filters[i], kernel_sizes[i], stride=strides[i], padding=paddings[i]))
                if BN:
                    self.net.add_module("BN-2d-" + str(i), nn.BatchNorm2d(number_of_filters[i]))
                self.net.add_module("relu-conv-" + str(i), nn.ReLU())
                self.net.add_module("maxpool-" + str(i), nn.MaxPool2d(maxpool[i], maxpool[i]))
        
        self.net.add_module("flatten1", nn.Flatten())
        
        fc_down = self.net(torch.rand(1, channel_numbers[0], img_shape, img_shape)).size()[1]
        print(fc_down)
        
        self.net.add_module("fc-0", nn.Linear(fc_down, self.fclayout[0]))
        if BN:
            print("BN3: ", self.fclayout[0])
            self.net.add_module("BN-1d-0", nn.BatchNorm1d(self.fclayout[0]))
        self.net.add_module("Drop-0", nn.Dropout(dropout_rate))
        self.net.add_module("relu-fc-0", nn.ReLU())
        
        for i in range(len(fclayout)):
            if i == len(self.fclayout)-1:
                self.net.add_module("fc-" + str(i+1), nn.Linear(fclayout[i], self.label_shape))
            else:
                self.net.add_module("fc-" + str(i+1), nn.Linear(fclayout[i], fclayout[i+1]))
                if BN:
                    print("BN: " + str(i+1), fclayout[i+1])
                    self.net.add_module("BN-1d-" + str(i+1), nn.BatchNorm1d(fclayout[i+1]))
                self.net.add_module("Drop-" + str(i+1), nn.Dropout(dropout_rate))
                self.net.add_module("relu-fc-" + str(i+1), nn.ReLU())
    
    def forward(self, x):
        x = self.net(x)
        return x

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight.data)

In [31]:
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
#model.add_module("fc", nn.Linear(512, 8))

def training(trainingloader, max_epochs, optimizer, model, validationloader=None, run_name="", l1_lambda = 0):
    
    scaler = torch.cuda.amp.GradScaler()
    
    metrics_tracker = pd.DataFrame(columns=['epoch', 'train_loss', 'train_accuracy','val_loss', 'val_accuracy'])
    epochs_list = []
    train_loss_list = []
    train_accuracy_list = []
    val_loss_list = []
    val_accuracy_list = []
    
    start.record()
    for epochs in range(max_epochs):
        # Training
        correct = 0
        total_labels = 0
        total_loss = 0
        for batch_idx, (data, target) in enumerate(trainingloader):
            optimizer.zero_grad()
        
            target = target.to(device)

            data = np.repeat(data[..., np.newaxis], 3, -1)
        
            prediction = model.forward(data[:, 0, :, : ,:].permute(0, 3, 1, 2).to(device))
            
            argmax_predicton = torch.argmax(prediction, dim = 1)

            with torch.cuda.amp.autocast():
                loss = nn.functional.cross_entropy(prediction, target)
            
            if l1_lambda != 0:
                l1_norm = sum(p.abs().sum() for p in model.parameters())

                loss = loss + l1_lambda * l1_norm

            #loss gets updated after each batch, so a total loss is better to see if model is improving
            total_loss += loss

            total_labels += target.size(0)
            correct += (argmax_predicton == target).sum().item()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        print(f'Epoch: [{epochs+1}/{max_epochs}], total train (sum of) Loss: {total_loss}, Train Acc: {correct} / {total_labels}')
        epochs_list.append(epochs+1)
        train_loss_list.append(total_loss.cpu().detach())
        train_accuracy_list.append(correct/total_labels)
        
    metrics_tracker = [{'epoch': epochs_list[i], 'train_loss': train_loss_list[i].item(), 'train_accuracy': train_accuracy_list[i], 'val_loss': val_loss_list[i].item(), 'val_accuracy': val_accuracy_list[i]} for i in range(max_epochs)]
    
    end.record()
    # Waits for everything to finish running
    torch.cuda.synchronize()

    print(start.elapsed_time(end)/1000/60, "min")
    torch.cuda.empty_cache()
    return pd.DataFrame(metrics_tracker)

In [32]:
params = {'batch_size': 64,
          'shuffle': True}
max_epochs = 200

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
        
#model = CNN()
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model = CNN(img_shape=28, label_shape=10, number_of_c_layers = 2, number_of_filters=[16,32], channel_numbers=[3,16], maxpool=[2,2], kernel_sizes=[1,1], strides=[1,1], paddings=[0,0], fclayout=[200,32])#, dropout_rate = dropout)

model.cuda()
model.apply(weights_init)
print(model)


train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=64, shuffle=True)

results = training(train_loader, max_epochs, torch.optim.SGD(model.parameters(),lr=0.01), model, train_loader, run_name="testt")

6272
Epoch: [1/200], total train (sum of) Loss: 390.8134460449219, Train Acc: 52550 / 60000
Epoch: [2/200], total train (sum of) Loss: 178.5899658203125, Train Acc: 56557 / 60000
Epoch: [3/200], total train (sum of) Loss: 135.96829223632812, Train Acc: 57350 / 60000
Epoch: [4/200], total train (sum of) Loss: 109.54197692871094, Train Acc: 57844 / 60000
Epoch: [5/200], total train (sum of) Loss: 92.8230972290039, Train Acc: 58136 / 60000
Epoch: [6/200], total train (sum of) Loss: 80.80097961425781, Train Acc: 58383 / 60000
Epoch: [7/200], total train (sum of) Loss: 70.76211547851562, Train Acc: 58610 / 60000
Epoch: [8/200], total train (sum of) Loss: 63.27030944824219, Train Acc: 58742 / 60000
Epoch: [9/200], total train (sum of) Loss: 55.80248260498047, Train Acc: 58874 / 60000
Epoch: [10/200], total train (sum of) Loss: 49.409366607666016, Train Acc: 59008 / 60000
Epoch: [11/200], total train (sum of) Loss: 44.17115783691406, Train Acc: 59120 / 60000
Epoch: [12/200], total train (sum 

KeyboardInterrupt: 